7.1 文字列として表現されているデータのタイプ

文字列データの種類：
1. カテゴリデータ
2. カテゴリデータを表す自由に書かれた文字列：　
3. 構造化された文字列
4. テキストデータ

カテゴリデータ：　
- カテゴリを表す文字列。ドロップダウンメニューの[red, blue, green]など。固定リストからのデータ。

カテゴリデータを表す自由に書かれた文字列：　
- 人に色の名前を自由に記入してもらったアンケート結果など
 - the xkcd Color Survey（https://blog.xkcd.com/2010/05/03/color-survey-results/ ）
- カテゴリ変数にエンコード必要（どれにも割り当てられないものは新カテゴリを作成したり、otherカテゴリにまとめるなど）
- 手作業必要（自動化困難）
- 可能なら自由記入形式ではなく選択式を強く推奨

構造化された文字列：
- 固定カテゴリに対応せず、ある構造を持った文字列（住所、郵便番号、人名、日付、電話番号、識別番号など）

テキストデータ：
- ツイート、チャットログ、レビュー、小説、Wikipediaの記事、電子書籍など

テキスト解析：
- コーパス：　データセット。文書の集合。
- 文書：　1つのテキストとして表現される個々のデータポイント

7.2 例題アプリケーション：映画レビューのセンチメント分析

データセット：
- 映画レビューのテキスト。内容が肯定的(pos)か否定的(neg)かで2クラス分類。

In [12]:
# インポート集
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mglearn
import os
%matplotlib inline

In [61]:
# 映画レビューのセンチメント分析
# データロード（http://ai.stanford.edu/~amaas/data/sentiment/）からダウンロ―ドして格納しておく）
from sklearn.datasets import load_files
ra = load_files("data/aclImdb/train/")
# 訓練テキスト、訓練ラベル作成
ta, ya = ra.data, ra.target
print(type(ta))
print(len(ta))
print(ta[1])
print(ya[1])

<class 'list'>
27449
b'i was disappointed. the film was a bit predictable and did not live up to the hype plastered all over the box. Having said that, the characters were well developed, the windego myth was used in a unique premise and the house was pretty spooky but it just missed for me. I kept waiting for that big AHHHHH or BOO! But it never came.<br /><br />Furthermore the movie was plagued with poor filming of poor special effects. Thus showing to much of a bad thing and not using atmosphere and viewer imagination to create the horror and suspense. Try movies like Session 9 or the Cube if your looking for a low-budget but well conceived horror movie.'
0


In [62]:
# -> レビュー数：27449件

In [63]:
# 改行をなくす（空白に置換）
ta = [doc.replace(b"<br />", b" ") for doc in ta]

In [64]:
np.bincount(ya)

array([12500, 12500,  2449], dtype=int64)

In [65]:
# テストデータも同様に処理
# データロード
re = load_files("data/aclImdb/test/")
# テストテキスト、テストラベル作成
te, ye = re.data, re.target
print(len(te))
print(np.bincount(ye))
# 改行をなくす（空白に置換）
te = [doc.replace(b"<br />", b" ") for doc in te]

25000
[12500 12500]


In [66]:
# 機械学習で扱えるように文字列を数字に変換する必要あり

7.3　Bag of Wordsによるテキスト表現

Bag of Words（BoW)：
- 言葉の袋。構文無視して単語の出現数だけ数える（コーパスの単語がテキストに出現する数をカウント）
- 計算手順：
 1. トークン分割：　個々の文書を単語（＝トークン）に分割。ホワイトスペース（スペース、改行、タブ)や句読点で句切る。
 2. ボキャブラリ構築：　全ての文書に現れる全ての単語をボキャブラリとして集め、番号を付ける（アルファベット順など）
 3. エンコード：　個々の文書に対してボキャブラリの単語が現れる回数を数える。

- 出力：　
 - 1文書1ベクトル。(1単語1特徴量(0/1)を割り当てた数値表現）
 - SciPyの疎行列。非ゼロ要素のみ格納（殆どの文書にはボキャブラリ中の単語のごく一部しか使われないため。省メモリ。）


In [68]:
# トイデータセット x BoW
# データ作成（2つの文書（データ点））
w = ["The fool doth think he is wise,",
     "but the wise man knows himself to be a fool"]
w

['The fool doth think he is wise,',
 'but the wise man knows himself to be a fool']

In [80]:
# CountVectorizer使用
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
# 適合（訓練データのトークン分割＋ボキャブラリ構築）
v.fit(w)
# ボキャブラリ
print(v.vocabulary_)
# ボキャブラリサイズ
print(len(v.vocabulary_))

{'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}
13


In [76]:
# BoW作成
bow = v.transform(w)
# bowの解説
print(repr(bow))
# bowの内容（SciPy疎行列をnumpy（蜜）行列に変換（実際には多すぎてメモリエラーの危険）。numpyは0もメモリに格納する）
print(bow.toarray())

<2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


In [81]:
# -> 2x13：　文書数2、ボキャブラリ13（全文書の重複なし単語数）を表す。SciPy疎行列。
#    2つのデータ点にそれぞれ行が割り当てられ、ボキャブラリ中の単語に各特徴量が割り当てられている。
#    bowの内容：　各単語の出現回数。今回の2文書では、同じ単語が最大1度しか使われていないため0か1のみ。
# 　　　　　　　　ボキャブラリの最初から、単語be：0回、but：0回、doth：1回の出現を表す。

In [82]:
# 映画レビューのBoW
v = CountVectorizer().fit(ta)
xa = v.transform(ta)
print(repr(xa))

<27449x77975 sparse matrix of type '<class 'numpy.int64'>'
	with 3762717 stored elements in Compressed Sparse Row format>


In [83]:
# -> 文書数27449、ボキャブラリ77975。

In [84]:
#
n = v.get_feature_names()
print(len(n))
print(n[:20])
print(n[20010:20030])
print(n[::2000])

77975
['00', '000', '0000000000001', '000000003', '00001', '00015', '000s', '001', '003830', '006', '007', '0079', '0080', '0083', '0093638', '00am', '00pm', '00s', '01', '01pm']
['dithers', 'dithyrambical', 'ditka', 'ditsy', 'ditties', 'ditto', 'dittrich', 'ditty', 'ditz', 'ditzy', 'diurnal', 'diva', 'divagations', 'divali', 'divas', 'dive', 'dived', 'diver', 'diverge', 'diverged']
['00', 'adulterously', 'apostrophe', 'balooned', 'blanketed', 'bulked', 'chamberlin', 'coloured', 'crapper', 'dekho', 'disturbs', 'eighth', 'exhooker', 'flavorings', 'gaspingly', 'groove', 'heroic', 'ilses', 'irritated', 'khoobsurat', 'lengths', 'magnesium', 'meneses', 'mousse', 'nosher', 'pack', 'pierce', 'proclivity', 'reaccounting', 'richandson', 'satyr', 'sherri', 'soderberghian', 'stoppers', 'taiwan', 'tolkiens', 'unbuckles', 'verbosity', 'wicked']


In [85]:
# →　特徴量数：77975
# 　　特徴量の最初の20個
# 　　特徴量の20010番から20030番まで
# 　　特徴量を2000個おきに取り出す（アルファベット順になっている）

# 意味的に非常に類似した単語や、単数形と複数形などは同じ特徴量としたい。

In [86]:
# CVスコア（ロジスティック回帰）
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
s = cross_val_score(LogisticRegression(), xa, ya, cv=5)
# CVスコアの平均
print(np.mean(s))

C:\Users\toa\Anaconda3\envs\aidemy3.6\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\toa\Anaconda3\envs\aidemy3.6\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8244379336583417


In [87]:
# -> 平均CVスコア88は普通

In [ ]:
# グリッドサーチ
from sklearn.model_selection import GridSearchCV
pg = {'C': [0.001, 0.01, 0.1, 1, 10]}
g = GridSearchCV(LogisticRegression(), pg, cv=5)
g.fit(xa, ya)
# ベストCVスコア
print(g.best_score_)
# ベストパラメタ
print(g.best_params_)
# テストスコア
xe = v.transform(te)
print(g.score(xe, ye))

p327